In [2]:
import pandas as pd
        
pure_isw = pd.read_csv('pure_isw.csv')
pure_isw

,text_for_vect2,date
0,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-25
1,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-26
2,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-27
3,"[ ' mason ' , ' clark ' , ' george ' , ' barro...",2022-02-28
4,"[ ' frederick ' , ' kagan ' , ' george ' , ' b...",2022-03-01
...,...,...
324,"[ ' karolina ' , ' hird ' , ' grace ' , ' mapp...",2023-01-21
325,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-22
326,"[ ' russian ' , ' offensive ' , ' campaign ' ,...",2023-01-23
327,"[ ' karolina ' , ' hird ' , ' riley ' , ' bail...",2023-01-24


In [3]:
!pip3 install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.3/23.3 MB 3.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gensim: filename=gensim-4.3.2-cp312-cp312-macosx_10_9_universal2.whl size=24462771 sha256=415d5fd7245616c75a205b8a20c091ac89a2c3a11a40e954ef20607b59c28c12
  Stored in directory: /Users/ivantyshchenko/Library/Caches/pip/wheels/50/c0/ac/7bb08954bc59d390c848b480a3fc5eec68c14bc77bf334d624
Successfully built gensim


In [11]:
import gensim
from gensim.utils import simple_preprocess

# Tokenize the strings
tokenized_text = [simple_preprocess(text) for text in pure_isw["text_for_vect2"]]

# Create a dictionary
dic = gensim.corpora.Dictionary(tokenized_text)

# Create a bag of words corpus
bow_corpus = [dic.doc2bow(doc) for doc in tokenized_text]

# Train LDA model
lda = gensim.models.LdaMulticore(bow_corpus, id2word=dic, num_topics=10, passes=15, workers=4)




In [10]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, bow_corpus, dictionary=dic)
vis 

/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
/Users/ivantyshchenko/Projects/Python/4-pythonic-squad/.venv/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.177017 -0.005140       1        1  43.016146
11    -0.271513  0.050650       2        1  19.778141
10    -0.216654  0.023236       3        1  13.792332
0     -0.265354  0.087975       4        1  11.399170
14    -0.204177 -0.117488       5        1   9.334596
9     -0.188951 -0.019249       6        1   1.983160
6      0.004811 -0.119599       7        1   0.358737
1      0.055223 -0.044931       8        1   0.162603
5      0.019093  0.070773       9        1   0.133858
2      0.208421  0.024035      10        1   0.021300
4      0.215473  0.008889      11        1   0.003991
7      0.205250  0.011524      12        1   0.003991
8      0.207560  0.009713      13        1   0.003991
12     0.199122  0.010296      14        1   0.003991
13     0.208712  0.009315      15        1   0.003991, topic_info=              Term          Freq         Total Category  logprob  loglift
312        russian  31363.000000  31363.000000  Default  30.0000  30.0000
369      ukrainian  16131.000000  16131.000000  Default  29.0000  29.0000
140          force  16737.000000  16737.000000  Default  28.0000  28.0000
216       military   4499.000000   4499.000000  Default  27.0000  27.0000
298         report   6812.000000   6812.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
337          staff      0.003291   2506.455501  Topic15  -8.8068  -3.4143
11            also      0.003255   2182.298085  Topic15  -8.8178  -3.2869
1475  mobilization      0.003218   1904.473852  Topic15  -8.8293  -3.1621
276       position      0.003206   1919.985391  Topic15  -8.8330  -3.1740
582        support      0.003256   2682.259147  Topic15  -8.8174  -3.4928

[1169 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
8412       5  0.869805      abide
5711       1  0.535596   accuracy
5711       8  0.535596   accuracy
10280      6  0.656320  adamantly
5479       1  0.060508   adamivka
...      ...       ...        ...
1233       4  0.050367       znpp
1233       5  0.050367       znpp
1233       6  0.057235       znpp
1233       7  0.002289       znpp
7298       3  0.948311       ztpp

[1921 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 12, 11, 1, 15, 10, 7, 2, 6, 3, 5, 8, 9, 13, 14])

In [12]:
import spacy 
nlp = spacy.load('en_core_web_sm')
